# Machine Learning Research Topic Modeling with BERTopic

This notebook demonstrates how to perform topic modeling on a dataset of Machine Learning research papers from arXiv using the BERTopic library. It covers data loading, model training (or loading a pre-trained model), topic visualization, and analysis.

## **1. Installation of Libraries**


Note that `%%capture` hides the output of the library installations. Remove it to see the output. 

Overview of installed dependencies:
*   **`bertopic`**: The core library for BERTopic-based topic modeling.
*   **`litellm`**: A library that simplifies LLM API calls by providing a single API client for any LLM provider (e.g. Gemini, Anthropic, OpenAI, AWS Bedrock, etc.)
*   **`sentence-transformers`**: Used for generating sentence embeddings, which are crucial for BERTopic's understanding of semantic meaning.
*   **`scikit-learn`**: provides machine learning tools, including CountVectorizer used here for text preprocessing.
*   **`pandas`**: Used for data manipulation and working with DataFrames. Not installed here because it comes 
*   **`torch`**: PyTorch is a deep learning framework, and we are checking for hardware acceleration.
*   **`kagglehub`**: Used to fetch data from Kaggle.

In [1]:
%pip install litellm bertopic scikit-learn kagglehub octis --quiet 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 90.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 83.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 48.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.

## 2. Importing Libraries and Preparing the Dataset

In [2]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from kaggle_secrets import UserSecretsClient
import pandas as pd
import torch
import kagglehub
import numpy as np

dataset = "/kaggle/input/arxiv-ml-ai-052023-052025/arxiv_ml_ai_papers_last_2_years.csv"

try:
    df = pd.read_csv(dataset)
except:
    dataset = kagglehub.dataset_download('student344/arxiv-ml-ai-052023-052025', path="/kaggle/input/arxiv-ml-ai-052023-052025/arxiv_ml_ai_papers_last_2_years.csv")
    df = pd.read_csv(dataset)
    
df["text"] = df["title"] + " " + df["summary"]
print("The dataset has been parsed successfully.")

The dataset has been parsed successfully.


After importing the required libraries, we combine each paper's title and abstract into a single text column for preprocessing. This combined text serves as input for our embedding model. 

We preserve most of the text, including stop words, since transformer-based embedding models require full contextual information to generate accurate embeddings. Light preprocessing is performed to remove escape characters, LaTeX code, URLs, and other noise. [As recommended by BERTopic's developers](https://maartengr.github.io/BERTopic/faq.html#how-do-i-remove-stop-words), any additional preprocessing steps are performed *after* generating the embeddings.



In [3]:
import re

def preprocess(text: str) -> str:
    # Remove inline LaTeX math expressions: $...$
    text = re.sub(r'\$(.*?)\$', '', text)
    
    # Remove display math
    text = re.sub(r'\$\$(.*?)\$\$', '', text, flags=re.DOTALL)
    text = re.sub(r'\\(.∗?)(.*?)\\', '', text, flags=re.DOTALL)

    # Remove common LaTeX commands (e.g., \cite{}, ??????????????????????????????\ref{}, \begin{}, etc.)
    text = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', text)
    # Remove LaTeX escape sequences such as \\% or \\_
    text = re.sub(r'\\([%_&#$])', r'\1', text)

    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    
    # Remove URLs (http/https)
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

docs = df['text'].apply(preprocess).to_list()
print('Data preprocessing complete.')

Data preprocessing complete.


## 3. Loading the Embeddings

Sentence embeddings are numerical representations of text that capture semantic meaning. This section handles loading embeddings from the current environment or downloading them from Kaggle. To replicate all steps to create the embeddings from the embedding model, we can set `load_embeddings_from_storage` to `False`. 

In [4]:
embeddings = None
load_embeddings_from_storage = False  # set to False to recreate the embeddings

def load_embeddings():
    try:
        print("Attempting to load embeddings from local storage...")
        # return np.load("/kaggle/input/machine-learning-arxiv-papers-122022-122024/embeddings.npy")
        return np.load("/kaggle/working/arxiv_gist_embeddings_small.npy")
    except Exception as e:
        print(f"Local load failed: {e}. Trying kagglehub download...")
        return kagglehub.dataset_download(
            'student344/machine-learning-arxiv-papers-122022-122024',
            path="arxiv_gist_embeddings.npy"
        )

if load_embeddings_from_storage:
    embeddings = load_embeddings()
    print("Embeddings loaded successfully.")
else:
    print("Skipping embedding load: will recreate embeddings.")


Skipping embedding load: will recreate embeddings.


### 3.1 Creating the Text Embeddings

*   **Embedding Model:** We use the "avsolatorio/GIST-Embedding-v0" model (via SentenceTransformers), one of the highest scoring semantic text embedding models of the <100m parameter range on the [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard), particularly on clustering-related benchmarks. This model has a 33.4 million parameter size, which is lightweight enough to load quickly in a Kaggle or Colab runtime (with the GPU runtime enabled).
*   **Encoding:** The `embedding_model.encode()` function generates embeddings for the `docs` (the list of paper texts).
*   **Device Usage:** `device=device` ensures that the embedding generation uses the available hardware acceleration (GPU or CPU).

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
# embedding_model_name = "prdev/mini-gte"
embedding_model_name = "avsolatorio/GIST-small-Embedding-v0"
model_prompt = "clustering"

embedding_model = SentenceTransformer(embedding_model_name,
                                      trust_remote_code=True, 
                                      device=device,
                                     )
if not load_embeddings_from_storage:
    print("Creating embeddings...")
    embeddings = embedding_model.encode(docs, show_progress_bar=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating embeddings...


Batches:   0%|          | 0/379 [00:00<?, ?it/s]

### 3.2 Creating the Topic Model

We are now ready to initialize a BERTopic model. 

#### Parameters
- Now that the embeddings are generated, CountVectorizer is used to remove English stop words (words like "for", "and", " "to", etc.).
- The previously defined embedding model is used.
- N-gram range of (1,3) is used to capture single words, bi-grams (pairs of words such as "Computer Vision" and "Reinforcement Learning"), and tri-grams (terms like "Large Language Models" "Time Series Forecasting", etc.)
- Verbose mode enabled for training progress updates.
- We set a minimum topic size of 35, so that smaller topics with less than 35 document examples do not get clustered. This prevents noise in the results at the expense of not capturing the entire breadth of topics in the dataset. 


In [6]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2))
print("Creating new model...")

topic_model = BERTopic(
    verbose=True,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    n_gram_range=(1, 3),
    min_topic_size=30,
)

topics, probs = topic_model.fit_transform(docs, embeddings)
print("The topic model has been created.")


2025-05-19 19:32:56,312 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Creating new model...


2025-05-19 19:33:22,048 - BERTopic - Dimensionality - Completed ✓
2025-05-19 19:33:22,049 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-19 19:33:22,541 - BERTopic - Cluster - Completed ✓
2025-05-19 19:33:22,553 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-19 19:33:29,351 - BERTopic - Representation - Completed ✓


The topic model has been created.


The following table shows the model output. We will explore this further in the sections below. 
- **Topic**: The topic ID. Note that the Topic ID "-1" represents the outliers (documents that were not clustered into any specific topic). 
- **Count**: The number of documents that were clustered into the topic.
- **Representation**: The list of the top words that represent the topic.
- **Representative_Docs**: A sample of representative documents for the topic.

In [7]:
topic_info = topic_model.get_topic_info()
topic_info.head(30)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3725,-1_data_models_learning_model,"[data, models, learning, model, based, methods...",[Pre-training on High Definition X-ray Images:...
1,0,628,0_language_models_llms_language models,"[language, models, llms, language models, larg...",[FreqKV: Frequency Domain Key-Value Compressio...
2,1,320,1_federated_fl_federated learning_clients,"[federated, fl, federated learning, clients, p...",[A Survey on Blockchain-Based Federated Learni...
3,2,303,2_rl_policy_reinforcement_reinforcement learning,"[rl, policy, reinforcement, reinforcement lear...",[Reinforcement Learning with Continuous Action...
4,3,297,3_networks_neural_gradient_optimization,"[networks, neural, gradient, optimization, neu...",[Beyond NTK with Vanilla Gradient Descent: A M...
5,4,289,4_traffic_driving_autonomous_trajectory,"[traffic, driving, autonomous, trajectory, veh...",[Risk-anticipatory autonomous driving strategi...
6,5,268,5_neural_equations_physics_pdes,"[neural, equations, physics, pdes, operator, p...",[Advancing Generalization in PINNs through Lat...
7,6,256,6_visual_multimodal_image_vision,"[visual, multimodal, image, vision, language, ...",[Unleashing Text-to-Image Diffusion Prior for ...
8,7,244,7_matrix_regression_learning_label,"[matrix, regression, learning, label, data, di...",[Regularized Linear Discriminant Analysis Usin...
9,8,230,8_graph_node_graphs_gnns,"[graph, node, graphs, gnns, graph neural, gnn,...",[Graph Condensation for Inductive Node Represe...


### Evaluation Metrics

Below, we use standard evaluation metrics for topic modeling. We use the OCTIS (Optimizing and Comparing Topic Models is Simple) library to calculate the NPMI topic coherence score and the topic diversity score. 
The coherence score calculation can take a few minutes, even when using multithreading.

The NPMI coherence score of **~0.260** suggests that our topics are reasonably interpretable, meaning the words within each topic tend to be semantically related. Our diversity score of **~0.70** suggests that our model is decently capturing different themes within our data, although an ideal score would be above 80. This is expected, however, since we have limited our topic size to a minimum of 35 representative documents (skipping many smaller topics), and our dataset is supposed to have common themes (topics related to machine learning).  

While it is good to have these evaluation metrics, determining the relevance of clustered topics is an inherently subjective process, so it is difficult and impractical to obtain a clear picture of the topic model's quality while only relying on purely quantitative metrics. 

In [8]:
topics_dict  = topic_model.get_topics()                 # one call
topic_words  = [
    [w for w, _ in words[:10]]                 # top-10 words
    for tid, words in topics_dict.items()
    if tid != -1                               # skip outliers
]


In [9]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

vectorizer = topic_model.vectorizer_model          # the CountVectorizer that c-TF-IDF used
analyzer   = vectorizer.build_analyzer()  # includes lowercase, n-grams, stop-words …

tokenized_docs = [analyzer(d) for d in docs]


In [10]:
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity

octis_format = {"topics": topic_words}

coh = Coherence(texts=tokenized_docs, topk=5, processes=4).score(octis_format)
print(f"c_npmi={coh:.3f}")

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


c_npmi=0.217


In [11]:
div = TopicDiversity(topk=10).score(octis_format)
print(f"diversity={div:.3f}")


diversity=0.731


## 4. Exploring and Visualizing the Results

### 4.1 Creating New Topic Labels and Summaries using Large Language Models

As seen in the table above, the names of our topics use a simple topic representation model: the topic ID number, followed by the top representative words, separated with underscores.
We can use a Large Language Model (LLM) to create a more descriptive label. Additionally, we can use it to generate a short summary for each topic. 

At the moment, Google offers an experimental version of Gemini Flash 2.0 at only 10 cents per million input tokens and 40 cents per million output tokens. Additionally, it offers 15 requests per minute (RPM) and 1500 free requests per day. It is distinguished for being small, fast, and capable of high quality outputs for non-reasoning tasks. Since the model has a large context window, we do not really need to batch our requests, but our code below processes our topics list in two halves, just in case (some models may have degraded performance when the included context is too large).

We use LiteLLM, a library that supports API calls to many LLM providers using a common interface. With this library, we can swap out our chosen model for a new one in the future without having to rewrite most of the code.

Note: You must set up your own Gemini API key. After creating your key through Google AI Studio, go to the "Add-ons" > "Secrets" > "Add Secret", then create a secret with the name "GEMINI_API_KEY" and paste your unique API token. 

In [12]:
import os, math, json, time
from litellm import completion
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GEMINI_API_KEY")

os.environ["GEMINI_API_KEY"] = api_key

def prompt_builder(df_chunk):
    # concise system instruction
    system_msg = """You are an ML researcher who names Machine Learning research topic clusters 
    generated by BERTopic. 
    For each topic you receive, return a JSON object with:
          id      : integer   (the topic ID)
          Label   : ≤ 6 words (concise title)
          Summary : ≤ 25 words (short description of the topic)
        Each topic cluster you receive includes its name and representation (top words and phrases found in the cluster).
        Respond with a JSON list only—no extra text.
    """.strip()

    # build a small TSV block the model can read
    rows = [
        f"\t{r.Name}\t{r.Representation}"
        for _, r in df_chunk.iterrows()
    ]
    user_msg = (
        "Columns: id, size, placeholder_label, top_keywords\n"
        "```text\n" + "\n".join(rows) + "\n```"
    )

    return [
        {"role": "system", "content": system_msg},
        {"role": "user",   "content": user_msg}
    ]

# split the DataFrame
midpoint = math.ceil(len(topic_info) / 2)

first_half  = topic_info.iloc[:midpoint]
second_half = topic_info.iloc[midpoint:]

# build prompts and call LLM
def call_gemini(df_part):
    messages = prompt_builder(df_part)      
    resp = completion(
        model="gemini/gemini-2.0-flash-exp",
        messages=messages,
        response_format={"type": "json_object"},
        temperature=0.3,                         
        max_tokens=100000,                    
    )
    raw_json = resp.choices[0].message.content   
    return json.loads(raw_json)                  

results_1 = call_gemini(first_half)
results_2 = call_gemini(second_half)

# --- 3. stitch back together --------------------------------------------------
all_results = results_1 + results_2
all_results.sort(key=lambda d: d["id"])

19:34:16 - LiteLLM:INFO: utils.py:2904 - 
LiteLLM completion() model= gemini-2.0-flash-exp; provider = gemini
19:34:25 - LiteLLM:INFO: utils.py:1211 - Wrapper: Completed Call, calling success_handler
19:34:25 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: gemini/gemini-2.0-flash-exp
19:34:25 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: gemini/gemini-2.0-flash-exp
19:34:25 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: gemini/gemini-2.0-flash-exp
19:34:25 - LiteLLM:INFO: utils.py:2904 - 
LiteLLM completion() model= gemini-2.0-flash-exp; provider = gemini
19:34:34 - LiteLLM:INFO: utils.py:1211 - Wrapper: Completed Call, calling success_handler
19:34:34 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: gemini/gemini-2.0-flash-exp
19:34:34 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: gemini/gemini-2.0-flash-exp


The model was instructed to generate its output in structured JSON, which can be easily parsed into a dataframe and merged with the dataframe generated by `get_topic_info()`.

In [13]:
import json

label_df = pd.DataFrame(all_results)                
label_df.rename(columns={"id": "Topic"}, inplace=True)

# apply to BERTopic 
# merge so we keep original ordering and any extra columns
topic_info = topic_model.get_topic_info()
topic_info = topic_info.merge(label_df, on="Topic", how="left")

# use the “label” column as custom topic names
custom_labels = topic_info.set_index("Topic")["Label"].to_dict()
topic_model.set_topic_labels(custom_labels)

# store the summaries for later display
topic_summaries = topic_info.set_index("Topic")["Summary"].to_dict()
print('Topic labels and summaries have been added to the topic model.')

Topic labels and summaries have been added to the topic model.


### 4.2 Top Topics Table

The table below shows the top 30 most frequent topics, including the new summaries and labels, with the outliers topic (Topic -1) at the top. 

In [14]:
topic_info[["Topic", "Count", "Name", "Label", "Summary"]].head(30)

,Topic,Count,Name,Label,Summary
0,-1,3725,-1_data_models_learning_model,General Machine Learning Methods,"General machine learning methods, models, and ..."
1,0,628,0_language_models_llms_language models,Large Language Model Training and Tuning,Training and fine-tuning large language models...
2,1,320,1_federated_fl_federated learning_clients,Federated Learning for Privacy,Federated learning (FL) techniques for privacy...
3,2,303,2_rl_policy_reinforcement_reinforcement learning,Reinforcement Learning and Policy Optimization,Reinforcement learning (RL) algorithms for opt...
4,3,297,3_networks_neural_gradient_optimization,Neural Network Optimization Techniques,Optimization methods for training neural netwo...
5,4,289,4_traffic_driving_autonomous_trajectory,Autonomous Driving and Traffic Prediction,Machine learning applications in autonomous dr...
6,5,268,5_neural_equations_physics_pdes,Physics-Informed Neural Networks for PDEs,Using neural networks to solve partial differe...
7,6,256,6_visual_multimodal_image_vision,Multimodal Visual-Language Models,Combining visual and language information in m...
8,7,244,7_matrix_regression_learning_label,Matrix Regression and Label Learning,Matrix regression techniques and label learnin...
9,8,230,8_graph_node_graphs_gnns,Graph Neural Networks,Graph neural networks (GNNs) for analyzing and...


### 4.3 Intertopic Distance Map

Our first visualization shows the relationships between topics in a 2D space. Topics that are closer together are semantically more similar. Since we set the labels generated by the LLM as our custom topic labels, they can now be used in the visualizations.


The map allows for interactive exploration. Upon hovering over the circles, the topic names and sizes are shown. Any area can be selected to zoom in for closer inspection. The size of each circle corresponds to the topic's prevalence in the dataset, making it easy to identify dominant themes.

This visualization provides a clear and intuitive overview of topic relationships, and allows us to see which topics have enough overlap to be merged if we want to trim down our number of topics even further. To see the topic labels, sizes and IDs, simply hover over each circle with the mouse. The slider can be used to highlight a specific topic.



In [15]:
topic_model.visualize_topics(custom_labels=True)

### 4.3 Topic Word Scores Bar Chart

This bar chart visualization highlights the top words associated with each topic identified by the BERTopic model. The topics are represented by their most representative terms, ranked by relevance scores. The length of the bars corresponds to the importance of each word in defining the topic. Note how the Reinforcement Learning topic is not just represented by 'reinforcement' and 'learning,' but also by related concepts such as 'reward' (the feedback signal that the algorithm seeks to maximize over time) and 'policy' (the strategy or mapping from states to actions that the algorithm learns).
This demonstrates how the BERTopic model’s underlying embeddings effectively capture the semantic relationships between terms and concepts

In [16]:
topic_model.visualize_barchart(custom_labels=True, height=300, width=385)

### 4.4 Topic Similarity Heatmap

The similarity matrix heatmap offers a good way to inspect the relationships between pairs of topics. Each row and column corresponds to a particular topic, and the color of each cell reflects the degree of semantic similarity between those two topics. Darker cells along the diagonal indicate higher self-similarity (a topic compared to itself), while off-diagonal cells reveal how related (or unrelated) different topics are.

From the heatmap, you can see which topics tend to cluster together. Topics that share conceptual ground, such as “Supervised Learning” and “Unsupervised Learning”, appear in regions of higher similarity, suggesting that the language used to describe them overlaps significantly. Conversely, less closely related topics (e.g., “Weather & Climate Prediction” vs. “Audio, Speech & Music Synthesis”) have lower similarity scores, appearing in lighter-colored cells. Note that the results below might differ from the examples described, due to the stochastic nature of the topic model and the LLM outputs.

In [17]:
topic_model.visualize_heatmap(custom_labels=True, top_n_topics=20)

## 5. Document-Level Visualizations

### 5.1 Visualize Documents with Hoverable Titles

The following visualization is a scatter plot where each point represents a document. This scatter plot visualizes the distribution of documents and their assigned topics in a two-dimensional space. Each point represents a document, and points are colored according to their topic. Labels indicate the general area of the plot where a particular topic is most prominent, showing how the BERTopic model clusters semantically similar documents together.

The documents are colored by their assigned topic. Hovering over a point shows the document's title. 

The clustering and separation of points indicate the effectiveness of the topic modeling process, with clear groupings suggesting coherent topic definitions.


In [18]:
topic_model.visualize_documents(df["title"], 
                           title="Documents and Topics",
                           embeddings=embeddings,
                           custom_labels=True, 
                           hide_annotations=True, 
                           topics=topics)

### 5.2 Documents with Labeled Topics

This version of the visualization shows the document clusters with their respective topic labels. In order to make space for the labels, only the top 55 document clusters are used for the `topics` parameter.

In [19]:
top_topics = topics[:55]
topic_model.visualize_documents(docs, 
                           title="Documents and Topics",
                           embeddings=embeddings, 
                           hide_document_hover=True, 
                           custom_labels=True, 
                           topics=top_topics)

## 6. Topic Search

Below, we put our topic modeling to use with a search engine for our data, allowing for filtering by topic. The engine uses a simple cosine similarity algorithm and leverages the same embedding model that was used for the topic model.

* Enter you search query and click the search button to search accross all topics.
* Click on a topic from the list to choose a filter.
* Click the clear button to remove your input text and your filtered topic.

Try the following search queries: "rag", "image segment", "TTS"

In [20]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output


def search_papers(
    query: str,
    topic_model,
    df,
    embeddings: np.ndarray,
    embedder,
    topic_info,
    top_k: int = 5,
    topic_filter: list[int] | None = None,
):
    """
    Return a list of the top‐k documents most similar to *query*.
    """
    # Encode the query
    query_emb = embedder.encode([query], show_progress_bar=False)[0]

    # Vector-normalized cosine similarity
    similarities = (
        np.dot(embeddings, query_emb)
        / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(query_emb) + 1e-9)
    )

    # Per-document topic labels
    # BERTopic exposes them through .topics_
    doc_topics = topic_model.topics_

    # Optional topic filter
    if topic_filter:
        mask = np.isin(doc_topics, topic_filter)
        similarities = similarities * mask

    # Rank results
    top_idx = similarities.argsort()[-top_k:][::-1]

    results = []
    for idx in top_idx:
        topic_id = doc_topics[idx]

        # Safe lookup – fall back to 'Unknown' if topic was deleted
        try:
            if topic_id == -1:
                topic_label = "Outlier Topics"
            else:
                topic_label = topic_info.loc[topic_info["Topic"] == topic_id, "Label"].iloc[0]
        except IndexError:
            topic_label = f"Topic {topic_id}"

        results.append(
            {
                "title": df.at[idx, "title"],
                "summary": df.at[idx, "summary"],
                "topic": topic_label,
                "topic_id": topic_id,
                "similarity": float(similarities[idx]),
            }
        )
    return results


def create_search_interface(
    topic_model,
    df,
    embeddings,
    embedder,
    topic_info,
):
    # Widgets
    search_box = widgets.Text(
        placeholder="Enter search query…",
        description="Search:",
        layout=widgets.Layout(width="50%"),
    )

    topic_options = [
        (row["Name"], int(row["Topic"]))
        for _, row in topic_info[topic_info["Topic"] != -1].iterrows()
    ]

    topic_dropdown = widgets.SelectMultiple(
        options=topic_options,
        description="Filter topics:",
        layout=widgets.Layout(width="50%", height="200px"),
    )

    results_out = widgets.Output()

    def run_search(_):
        with results_out:
            clear_output()
            if not search_box.value.strip():
                display(HTML("<em>Please enter a search query.</em>"))
                return

            topic_filter = list(topic_dropdown.value) or None
            hits = search_papers(
                search_box.value,
                topic_model,
                df,
                embeddings,
                embedder,
                topic_info,
                top_k=15,
                topic_filter=list(topic_dropdown.value) or None,
            )
            if not hits:
                display(HTML("<em>No results.</em>"))
                return

            for i, hit in enumerate(hits, 1):
                html = f"""
                <div style="margin:12px 0;padding:10px;border:1px solid #ddd;">
                  <h3>{i}. {hit['title']}</h3>
                  <p><b>Topic:</b> {hit['topic']} (ID {hit['topic_id']})</p>
                  <p><b>Similarity:</b> {hit['similarity']:.3f}</p>
                  <p><b>Summary:</b> {hit['summary'][:200]}…</p>
                </div>"""
                display(HTML(html))

    def clear_form(_):
        search_box.value = ""
        topic_dropdown.value = ()
        with results_out:
            clear_output()

    # Assemble UI
    search_btn = widgets.Button(description="Search")
    clear_btn = widgets.Button(description="Clear")
    search_btn.on_click(run_search)
    clear_btn.on_click(clear_form)

    ui = widgets.VBox(
        [
            search_box,
            topic_dropdown,
            widgets.HBox([search_btn, clear_btn]),
            results_out,
        ]
    )
    display(ui)


# Launch
create_search_interface(topic_model, df, embeddings, embedding_model, topic_info)
